In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import pandas as pd

### Use this github directory path version for submission.
### superset_df_long = pd.read_csv("data/stage_2/superset_long.csv")
superset_df = pd.read_csv("drive/Shareddrives/Datasets/superset_long.csv")
superset_df

,countyFIPS,State,County Name,StateFIPS,population,Date,Deaths,Cases
0,1001,AL,Autauga County,1,55869,2020-01-22,0,0
1,1003,AL,Baldwin County,1,223234,2020-01-22,0,0
2,1005,AL,Barbour County,1,24686,2020-01-22,0,0
3,1007,AL,Bibb County,1,22394,2020-01-22,0,0
4,1009,AL,Blount County,1,57826,2020-01-22,0,0
...,...,...,...,...,...,...,...,...
1800361,56037,WY,Sweetwater County,56,42343,2021-08-16,45,5127
1800362,56039,WY,Teton County,56,23464,2021-08-16,11,4068
1800363,56041,WY,Uinta County,56,20226,2021-08-16,14,2593
1800364,56043,WY,Washakie County,56,7805,2021-08-16,26,967


## Generate Weekly Statistics for a number of new cases and deaths across a State

Adding datetime to the dataframe to perform time series data operations, and finding data for the state of Pennslyvania

In [83]:
datetime_df = pd.DataFrame(pd.to_datetime(superset_df['Date'])) # Converting Date to datetime and putting it into a dataframe.
superset_undated = pd.DataFrame(superset_df.drop(columns=['Date'])) # Removing Date column from the superset dataframe.
superset_datetime = pd.concat([datetime_df.reset_index(drop=True), superset_undated.reset_index(drop=True)], axis=1).fillna(0) # Merge.
superset_sorted = superset_datetime.sort_values(by=['countyFIPS', 'Date']) # Sorting by countyFIPS so that Date has daily ordering. 

### This returns negative values. It should NOT. Something errorous is afoot.
superset_sorted['New_cases_per_day'] = superset_sorted.groupby('countyFIPS').Cases.diff() # Daily differences set new a new column.
superset_sorted['New_deaths_per_day'] = superset_sorted.groupby('countyFIPS').Deaths.diff()
superset_sorted['New_cases_per_day'] = np.where(superset_sorted['New_cases_per_day'] < 0, 0, superset_sorted['New_cases_per_day']) # Setting the negative values to zero. 
superset_sorted['New_deaths_per_day'] = np.where(superset_sorted['New_deaths_per_day'] < 0, 0, superset_sorted['New_deaths_per_day'])

PA_df = superset_sorted[superset_sorted['State'] == 'PA']
PA_df = PA_df.dropna()
PA_df

,Date,countyFIPS,State,County Name,StateFIPS,population,Deaths,Cases,New_cases_per_day,New_deaths_per_day
5386,2020-01-23,42001,PA,Adams County,42,103009,0,0,0.0,0.0
8528,2020-01-24,42001,PA,Adams County,42,103009,0,0,0.0,0.0
11670,2020-01-25,42001,PA,Adams County,42,103009,0,0,0.0,0.0
14812,2020-01-26,42001,PA,Adams County,42,103009,0,0,0.0,0.0
17954,2020-01-27,42001,PA,Adams County,42,103009,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1786966,2021-08-12,42133,PA,York County,42,449058,840,48177,98.0,0.0
1790108,2021-08-13,42133,PA,York County,42,449058,840,48267,90.0,0.0
1793250,2021-08-14,42133,PA,York County,42,449058,840,48373,106.0,0.0
1796392,2021-08-15,42133,PA,York County,42,449058,840,48492,119.0,0.0


Aggregate daily data to get the statistics for the entire week.

In [87]:
PA_weekly = PA_df.groupby([pd.Grouper(key='Date', freq='w'),'County Name'])['New_cases_per_day','New_deaths_per_day'].agg({'New_cases_per_day':'mean', 'New_deaths_per_day':'mean'}).reset_index().round(decimals=0)

week_cases_mean = weekly_PA['New_cases_per_day'].mean()
week_cases_median = weekly_PA['New_cases_per_day'].median()
week_cases_mode = weekly_PA['New_cases_per_day'].mode()
print(week_cases_mean,week_cases_median,week_cases_mode)
week_deaths_mean = weekly_PA['New_deaths_per_day'].mean()
week_deaths_median = weekly_PA['New_deaths_per_day'].median()
week_deaths_mode = weekly_PA['New_deaths_per_day'].mode()
print(week_deaths_mean,week_deaths_median,week_deaths_mode)

PA_weekly

32.37385362344902 6.0 0    0.0
dtype: float64
0.6752382664988311 0.0 0    0.0
dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Date,County Name,New_cases_per_day,New_deaths_per_day
0,2020-01-26,Adams County,0.0,0.0
1,2020-01-26,Allegheny County,0.0,0.0
2,2020-01-26,Armstrong County,0.0,0.0
3,2020-01-26,Beaver County,0.0,0.0
4,2020-01-26,Bedford County,0.0,0.0
...,...,...,...,...
5556,2021-08-22,Washington County,13.0,0.0
5557,2021-08-22,Wayne County,3.0,0.0
5558,2021-08-22,Westmoreland County,43.0,0.0
5559,2021-08-22,Wyoming County,5.0,0.0


## Compare the data against other states. (normalize by population)

Take a subset of the superset including the states Pennslyvania, Indiana, North Carolina, Iowa, Mississippi, Maine, and Nevada.

In [77]:
#### Use the below line for github dirs
#### superset_df_long = pd.read_csv("data/stage_2/superset_long.csv")
superset = pd.read_csv("drive/Shareddrives/Datasets/superset_long.csv")
superset['Date'] = pd.to_datetime(superset['Date'])
superset_sorted = superset.sort_values(by=['countyFIPS', 'Date'])



# Making columns valued the difference of last cases/death value for a County
##superset_sorted['New_cases_per_day'] = superset_sorted.groupby('County Name').Cases.diff()
##superset_sorted['New_deaths_per_day'] = superset_sorted.groupby('County Name').Deaths.diff()

##superset_sorted.loc[superset_sorted['New_cases_per_day'] < 0]

# NOMINALIZATION
##superset_nomin = superset_prenomin.dropna()
##superset_nomin['Normalized_new_cases_per_day'] = ((superset_nomin['New_cases_per_day'] - 0) / (superset_nomin['population'] - 1)) * 100000
##superset_nomin['Normalized_new_deaths_per_day'] = ((superset_nomin['New_deaths_per_day'] - 0)/ (superset_nomin['population'] - 1)) * 100000


##superset_prenomin.loc[superset_prenomin['New_cases_per_day'] < 0]
#minvalue = superset_nomin['Normalized_new_cases_per_day'].min()
#minvalue = superset_prenomin['New_cases_per_day'].min()
#minvalue

In [78]:
IN_df = superset_nomin[superset_nomin['State'] == 'IN']

# Making dataframe of Date, County Name, and Normalized data (average weekly new case/death count)
weekly_IN = IN_df.groupby([pd.Grouper(key='Date', freq='w'),'County Name'])['New_cases_per_day','New_deaths_per_day'].agg({'New_cases_per_day':'mean', 'New_deaths_per_day':'mean'}).reset_index().round(decimals=0)
weekly_IN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


KeyError: ignored

Fill column with weekly data based on the nominalized daily values for deaths and cases.
Making a dataframe to base the plotting on. 

In [ ]:
subset_weekly = superset_df_long_nomin.groupby([pd.Grouper(key='Date', freq='w'),'County Name','StateFIPS'])['Normalized_new_cases_per_day','Normalized_new_deaths_per_day'].agg({'Normalized_new_cases_per_day':'mean','Normalized_new_deaths_per_day' :'mean'}).reset_index().round(decimals=0)
state_weekly = pd.DataFrame(columns = ('Date', 'State', 'Mean_Nomalized_new_cases_week', 'Mean_Nomalized_new_deaths_week', 'Median_Nomalized_new_cases_week', 'Median_Nomalized_new_deaths_week','Mode_Nomalized_new_cases_week', 'Mode_Nomalized_new_deaths_week'))
subset_weekly

In [ ]:
#subset_weekly = pd.DataFrame(pd.to_datetime(subset_weekly['Date']))
#subset_weekly = pd.DataFrame(subset_weekly.drop(columns=['Date']))

for i in subset_weekly['StateFIPS'].unique():
    mean_cases = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_cases_per_day'].mean()
    mean_deaths = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_deaths_per_day'].mean()
    median_cases = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_cases_per_day'].median()
    median_deaths = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_deaths_per_day'].median()
    mode_cases = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_cases_per_day'].mode()
    mode_deaths = subset_weekly[subset_weekly['StateFIPS'] == i]['Normalized_new_deaths_per_day'].mode()
    Data = { 'StateFIPS': i, 'Mean_Nomalized_new_cases_week': mean_cases, 'Mean_Nomalized_new_deaths_week': mean_deaths, 'Median_Nomalized_new_cases_week': median_cases, 'Median_Nomalized_new_deaths_week': median_deaths, 'Mode_Nomalized_new_cases_week': mode_cases, 'Mode_Nomalized_new_deaths_week': mode_deaths}
    state_weekly = state_weekly.append(Data,ignore_index=True)

state_weekly

In [ ]:
state_weekly['Mode_Nomalized_new_cases_week'] = state_weekly['Mode_Nomalized_new_cases_week'].astype(str)
state_weekly['Mode_Nomalized_new_deaths_week'] = state_weekly['Mode_Nomalized_new_deaths_week'].astype(str)
state_weekly['Mode_Nomalized_new_cases_week'] = state_weekly['Mode_Nomalized_new_cases_week'].str[:4]
state_weekly['Mode_Nomalized_new_deaths_week'] = state_weekly['Mode_Nomalized_new_deaths_week'].str[:4]
state_weekly

In [ ]:

#statesData = pd.concat([statesdates.reset_index(drop=True),statesData.reset_index(drop=True)], axis=1).fillna(0)

#state_weekly.plot(x = 'StateFIPS', y=['Mean_Nomalized_new_cases_week','Median_Nomalized_new_cases_week','Mode_Nomalized_new_cases_week'], kind='bar',figsize=(12,12))
#px.show("png")
#state_weekly = px.data.tips()
#statedata = px.parallel_categories(state_weekly, color="size", color_continuous_scale=px.colors.sequential.Inferno)
#statedata.show()
state_weekly.REF
plt.hist([state_weekly['Mean_Nominalized_new_cases_per_week'].values, state_weekly['REF'].values], stacked=True)

In [ ]:
weeklyDataNC = weeklyData.loc[weeklyData['State'] == 'NC']

NCarolina_weeklyCases_mean = weeklyDataNC['New_Cases_Per_Day'].mean()
NCarolina_weeklyCases_median = weeklyDataNC['New_Cases_Per_Day'].median()
NCarolina_weeklyCases_mode = weeklyDataNC['New_Cases_Per_Day'].mode()

print('Weekly Cases Mean: ', NCarolina_weeklyCases_mean)
print('Weekly Cases Median: ', NCarolina_weeklyCases_median)
print('Weekly Cases Mode: ', NCarolina_weeklyCases_mode)

In [ ]:
NCarolina_weeklyDeaths_mean = weeklyDataNC['New_Deaths_Per_Day'].mean()
NCarolina_weeklyDeaths_median = weeklyDataNC['New_Deaths_Per_Day'].median()
NCarolina_weeklyDeaths_mode = weeklyDataNC['New_Deaths_Per_Day'].mode()

print('Weekly Deaths Mean: ', NCarolina_weeklyDeaths_mean)
print('Weekly Deaths Median: ', NCarolina_weeklyDeaths_median)
print('Weekly Deaths Mode: ', NCarolina_weeklyDeaths_mode)